In [1]:
import warnings
warnings.filterwarnings('ignore')

import json
import os
import string
import time

import pandas as pd
import numpy as np

from pathlib import Path

In [ ]:
json_dir = Path(r"../data/dvlog_text")
annotations_file = Path(r"../DVlog/dataset/dvlog_labels_v2.csv")

## Word2vec (Google-news)

In [ ]:
run_word2vec = True



## BioWordVec

## BioWordSent